In [ ]:
%load_ext lab_black

import math
import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import dask.dataframe as dd

# 给定一个半径为r的圆，AB是一条随机选出的弦，求其长度大于圆内接等边三角形边长sqrt(3)r的概率P

In [4]:
# 根据(x1,y1)和(x2,y2)两点计算直线方程
def lineEquationByDots(x1, y1, x2, y2):
    a = y2 - y1
    b = x1 - x2
    c = x2 * y1 - x1 * y2
    return a, b, c

In [5]:
# 根据点(x1,y1)和直线与X轴的夹角theta计算直线方程
# 当theta接近0度或者90度的时候，计算出来的结果会有无限接近0的情况
def lineEquationByDotAndDegree(x1, y1, theta):
    x2, y2 = np.cos(theta) + x1, np.sin(theta) + y1
    return lineEquationByDots(x1, y1, x2, y2)

In [6]:
# 点(X0,Y0)到直线 ax + by + c = 0的距离公式
def pointDistanceToline(a, b, c, X0, Y0):
    return abs(a * X0 + b * Y0 + c) / math.sqrt(a * a + b * b)

In [7]:
# 直线 ax + by + c = 0 和圆 x*x + y*y = r*r 相交的弦长平方公式
def chordSquaredValue(a, b, c, r):
    return (r * r - c * c / (a * a + b * b)) * 2

In [8]:
# 计算直线 ax + by + c = 0 和 圆 x*x + y*y = 1 的交点
def lineCircleDots(a, b, c):
    sqrt = (2 * a * c) * (2 * a * c) - 4 * (a * a + b * b) * (c * c - b * b)
    if sqrt < 0:
        return None
    sqrt = math.sqrt(sqrt)
    x1 = (-2 * a * c + sqrt) / (2 * (a * a + b * b))
    x2 = (-2 * a * c - sqrt) / (2 * (a * a + b * b))
    y1 = -1 * (x1 * a + c) / b
    y2 = -1 * (x2 * a + c) / b
    return x1, y1, x2, y2

In [9]:
# 计算点(x1,y1)到点(x2,y2)的距离
def chordLength(x1, y1, x2, y2):
    res = math.sqrt((x2 - x1) * (x2 - x1) + (y2 - y1) * (y2 - y1))
    return res

In [10]:
# 在 maxX, maxY的区域里随机选取一个点、以及随机一个夹角，计算概率
def simulateBySelectOnePointAndDegree(
    totalTryTimes, maxX, maxY, shouldPointInCircle=False
):
    r = 1.0
    targetChordLengthSqure = 3 / 2  # (sqrt(3)/2) * (sqrt(3)/2) * 2
    pCount = 0
    hitCount = 0
    falseCount = 0
    for i in range(totalTryTimes):
        x1 = random.uniform(-1.0 * maxX, 1.0 * maxX)
        y1 = random.uniform(-1.0 * maxY, 1.0 * maxY)
        if shouldPointInCircle == True:
            if x1 * x1 + y1 * y1 > r * r:
                continue
        pCount += 1
        theta = random.uniform(0, math.pi)
        a, b, c = lineEquationByDotAndDegree(x1, y1, theta)
        chordSqureResult = chordSquaredValue(a, b, c, r)
        if chordSqureResult >= targetChordLengthSqure:
            hitCount += 1
        if chordSqureResult < 0:
            falseCount += 1
    #     print(
    #         f"循环{totalTryTimes}次, \
    # 采样{pCount}次, \
    # 满足条件{hitCount}次, \
    # 不相交{falseCount}次, \
    # 概率1: {hitCount/pCount} , \
    # 概率2: {hitCount/(pCount-falseCount)}"
    #     )
    return (
        totalTryTimes,
        pCount,
        hitCount,
        falseCount,
        hitCount / pCount,
        hitCount / (pCount - falseCount),
    )


# 选取 x轴上的点，计算每一点的概率
def calcPointProb(totalTryTimes, x1, y1):
    r = 1.0
    targetChordLengthSqure = 3 / 2  # (sqrt(3)/2) * (sqrt(3)/2) * 2
    pCount = 0
    hitCount = 0
    falseCount = 0
    for i in range(totalTryTimes):
        pCount += 1
        theta = random.uniform(0, math.pi)
        a, b, c = lineEquationByDotAndDegree(x1, y1, theta)
        chordSqureResult = chordSquaredValue(a, b, c, r)
        if chordSqureResult >= targetChordLengthSqure:
            hitCount += 1
        if chordSqureResult < 0:
            falseCount += 1
    return (
        totalTryTimes,
        pCount,
        hitCount,
        falseCount,
        hitCount / pCount,
        hitCount / (pCount - falseCount),
    )

In [11]:
# 在 maxX, maxY的区域里采样，一个点在圆内计数/两个点都在圆内计数
def simulateBySelectRandom2Points(
    totalTryTimes,
    maxX,
    maxY,
    atleastOnePointInCircle=False,
    atleastTwoPointInCircle=False,
):
    r = 1.0
    targetChordLengthSqure = 3 / 2  # (sqrt(3)/2) * (sqrt(3)/2) * 2
    pCount = 0
    hitCount = 0
    falseCount = 0
    for i in range(totalTryTimes):
        x1 = random.uniform(-1.0 * maxX, 1.0 * maxX)
        y1 = random.uniform(-1.0 * maxY, 1.0 * maxY)
        x2 = random.uniform(-1.0 * maxX, 1.0 * maxX)
        y2 = random.uniform(-1.0 * maxY, 1.0 * maxY)
        # 两个点的任何一个位于圆外则跳过采样计数
        if atleastTwoPointInCircle == True:
            if x1 * x1 + y1 * y1 > r * r:
                continue
            if x2 * x2 + y2 * y2 > r * r:
                continue
        # 两个点都在圆外则跳过采样计数
        if atleastOnePointInCircle == True:
            if x1 * x1 + y1 * y1 > r * r and x2 * x2 + y2 * y2 > r * r:
                continue
        # 开始采样计数
        pCount += 1
        a, b, c = lineEquationByDots(x1, y1, x2, y2)
        chordSqureResult = chordSquaredValue(a, b, c, r)
        if chordSqureResult >= targetChordLengthSqure:
            hitCount += 1
        if chordSqureResult < 0:
            falseCount += 1
        pass
    print(
        f"循环{totalTryTimes}次, \
采样{pCount}次, \
满足条件{hitCount}次, \
不相交{falseCount}次, \
概率1: {hitCount/pCount} , \
概率2: {hitCount/(pCount-falseCount)}"
    )
    pass

In [12]:
# 蒙特卡洛模拟每一点的概率
def dotProbPDF(start, end, sampleCount, trytimes):
    resDf = pd.DataFrame(np.linspace(start, end, sampleCount), columns=["X"])
    ddf = dd.from_pandas(resDf, npartitions=6)
    ddf["Y"] = ddf.apply(
        lambda tmprow: calcPointProb(trytimes, tmprow["X"], 0)[-2],
        meta=("Y", "float64"),
        axis=1,
    )
    return ddf.compute()


# 数学公式推导的概率
def dotProbPDF2(start, end, sampleCount):
    resDf = pd.DataFrame(np.linspace(start, end, sampleCount), columns=["X"])
    ddf = dd.from_pandas(resDf, npartitions=6)
    ddf["Y"] = ddf.apply(
        lambda tmprow: math.asin(1.0 / (tmprow["X"] * 2)) / math.pi * 2,
        meta=("Y", "float64"),
        axis=1,
    )
    return ddf.compute()


# 计算从0 - 无穷大处的概率
def ConditionalPDF(distance):
    if distance <= 0.5:
        return 1
    if distance <= 1:
        return math.asin(1.0 / (distance * 2)) / math.pi * 2
    return math.asin(1.0 / (distance * 2)) / math.asin(1.0 / distance)


# 数学公式推到的条件概率
# P = P(弦长大于根号3) / P(直线与圆相交)
def dotProbPDF3(start, end, sampleCount):
    resDf = pd.DataFrame(np.linspace(start, end, sampleCount), columns=["X"])
    ddf = dd.from_pandas(resDf, npartitions=6)
    ddf["Y"] = ddf.apply(
        lambda tmprow: ConditionalPDF(tmprow["X"]),
        meta=("Y", "float64"),
        axis=1,
    )
    return ddf.compute()

In [ ]:
np.seterr("raise")
# xs, ys = dotProbPDF(0,100,1000,100000)
dfres = dotProbPDF(0.5, 1, 1000, 100000)

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(dfres["X"], dfres["Y"])
plt.show()

In [ ]:
dfres = dotProbPDF2(0.5, 1, 100)
plt.figure(figsize=(16, 6))
plt.plot(dfres["X"], dfres["Y"])
plt.show()

In [ ]:
dfres = dotProbPDF3(0, 5, 10000)
plt.figure(figsize=(10, 6))
plt.plot(dfres["X"], dfres["Y"])
plt.show()

In [ ]:
simulateBySelectRandom2Points(10000000, 100, 100, False, False)

In [ ]:
simulateBySelectRandom2Points(1000000, 5, 5, True, True)

In [ ]:
simulateBySelectRandom2Points(1000000, 5, 5, True, False)

In [ ]:
simulateBySelectOnePointAndDegree(10000000, 5, 5, False)